# Chains

Base course works with old apis of the library (LLMChain, SimpleSequentialChain, SequentialChain, LLMRouterChain & MultiPromptChain). However, this notebook will work with the suggested classes at the date of this was written (2025-08-15).

Create base LLM to use in the exercises

In [1]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model='llama3.2:1b', temperature=0.9)

## Basic Chain

A linear process following the order: `User input -> Prompt -> LLM -> Output`

In [2]:

from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("What is the best name to describe a company that makes {product}?")

chain = prompt | llm

result = chain.invoke({ "product": "Queen Size Sheet Sets" })
print(result.content)

For a company that produces queen-sized sheet sets, you may want to consider a name that reflects their product offerings, target market, or brand identity. Here are some suggestions:

1. **Comfort Keepers**: This name suggests that the company provides products to help customers keep their comfort levels high.
2. **Sheet Society**: This name implies that the company is part of a community of people who care about getting the best sheets possible.
3. **Bedside Essentials**: This name emphasizes the importance of having essential bedding and sheet sets in one's bedroom.
4. **Soft & Secure Sheet Co.**: This name highlights the comfort and security provided by the company's products.
5. **Dream Weavers Bedding**: This name suggests that the company is involved in creating a good night's sleep, with its products helping customers weave a restful dream.
6. **Restful Slumber Bedding**: This name reinforces the idea of getting a great night's sleep and helps customers associate your brand wit

## Sequential Chain

Used when the process has intermediante output. Initial input is processed and the corresponding output is the input of the following step

In [3]:
from langchain.schema.output_parser import StrOutputParser

first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}? Just return the name, nothing else."
)

second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)

chain_test = (
    first_prompt           # 1. Creates prompt asking for company name
    | llm                 # 2. Sends prompt to language model
    | StrOutputParser()   # 3. Extracts just the text content from the AI message
    | {"company_name": lambda x: x}  # 4. Wraps string in dict with key "company_name"
    | second_prompt       # 5. Uses dict to fill second prompt template
    | llm                 # 6. Sends second prompt to LLM
    | StrOutputParser()   # 7. Extracts just the text content from the AI message
)

chain_test.invoke({ "product": "eco-friendly packaging" })

'"GreenPack, an eco-friendly packaging solutions provider, offering sustainable and cost-effective alternatives to traditional shipping materials worldwide."'

Under some circumstances, one LLM may receive multiple inputs. This is also possible to handle as explained below:

In [5]:
from langchain.schema.runnable import RunnableParallel

translation_prompt = ChatPromptTemplate.from_template("Translate the following review to english:\n\n{Review}")

language_prompt = ChatPromptTemplate.from_template("What language is the following review:\n\n{Review}\n\nReturn ONLY the name of the language in English.")

followup_prompt = ChatPromptTemplate.from_template("Write a follow up response in {language} to the following review: {english_summary}")

translation_chain = translation_prompt | llm | StrOutputParser()
language_chain = language_prompt | llm | StrOutputParser()
followup_chain = followup_prompt | llm | StrOutputParser()

overall_chain = RunnableParallel({
    "english_summary": translation_chain,
    "language": language_chain,
}) | {
    "followup_message": followup_chain
}

review = "El menú es confuso y el control remoto está mal diseñado."
print(f"Review: {review}")
followup_message = overall_chain.invoke({"Review": review})["followup_message"]
print(f"Follow-up Message: {followup_message}")

Review: El menú es confuso y el control remoto está mal diseñado.
Follow-up Message: Aquí te dejo una posible respuesta en español:

"No te preocupes, hemos aprendido de nuestros errores y estamos trabajando para mejorar la experiencia del cliente. Vamos a optimizar el menú y diseñar un control remoto más práctico y fácil de usar. ¡Gracias por tu feedback!"


Another use case is when it is needed to route the output to a specific LLM depending on a condition. This can be done in the way exposed below:

In [6]:
from typing import Literal

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig
from langgraph.graph import END, START, StateGraph
from typing_extensions import TypedDict

Let's generate chains for subject matter experts in Physics, Math and History

In [7]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know."""

physics_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", physics_template),
        ("human", "{input}"),
    ]
)

physics_chain = physics_prompt | llm | StrOutputParser()

In [8]:
math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question."""

math_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", math_template),
        ("human", "{input}"),
    ]
)

math_chain = math_prompt | llm | StrOutputParser()

In [9]:
history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements"""

history_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", history_template),
        ("human", "{input}"),
    ]
)

history_chain = history_prompt | llm | StrOutputParser()

In [10]:
default_prompt = ChatPromptTemplate.from_template("{input}")

default_chain = default_prompt | llm | StrOutputParser()

Now let's create the router chain by using LangGraph

In [11]:
route_system = "Route the user's query to either the physics, math, history, or default expert."
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", route_system),
        ("human", "{input}"),
    ]
)


# Define schema for output:
class RouteQuery(TypedDict):
    """Route query to destination expert."""

    destination: Literal["physics", "math", "history", "default"]


route_chain = route_prompt | llm.with_structured_output(RouteQuery)

# For LangGraph, we will define the state of the graph to hold the query,
# destination, and final answer.
class State(TypedDict):
    query: str
    destination: RouteQuery
    answer: str


# We define functions for each node, including routing the query:
async def route_query(state: State, config: RunnableConfig):
    destination = await route_chain.ainvoke(state["query"], config)
    return {"destination": destination}


# And one node for each prompt
async def physics_prompt(state: State, config: RunnableConfig):
    return {"answer": await physics_chain.ainvoke(state["query"], config)}

async def math_prompt(state: State, config: RunnableConfig):
    return {"answer": await math_chain.ainvoke(state["query"], config)}

async def history_prompt(state: State, config: RunnableConfig):
    return {"answer": await history_chain.ainvoke(state["query"], config)}

async def default_prompt(state: State, config: RunnableConfig):
    return {"answer": await default_chain.ainvoke(state["query"], config)}

# We then define logic that selects the prompt based on the classification
def select_node(state: State) -> Literal["physics_prompt", "math_prompt", "history_prompt", "default_prompt"]:
    if state["destination"] == "physics":
        return "physics_prompt"
    if state["destination"] == "math":
        return "math_prompt"
    elif state["destination"] == "history":
        return "history_prompt"
    else:
        return "default_prompt"

Below the graph creation with the previous setup:

In [12]:
graph = StateGraph(State)

graph.add_node("route_query", route_query)
graph.add_node("physics_prompt", physics_prompt)
graph.add_node("math_prompt", math_prompt)
graph.add_node("history_prompt", history_prompt)
graph.add_node("default_prompt", default_prompt)

graph.add_edge(START, "route_query")
graph.add_conditional_edges("route_query", select_node)
graph.add_edge("physics_prompt", END)
graph.add_edge("math_prompt", END)
graph.add_edge("history_prompt", END)
graph.add_edge("default_prompt", END)

app = graph.compile()

Start asking the questions:

In [13]:
async def ask_question(query: str) -> None:
  """Ask a question and get an answer."""
  print(f"Question: {query}")
  result = await app.ainvoke({"query": query})
  print(f"Destination: {result['destination']}")
  print(f"Answer: {result['answer']}")

In [26]:
await ask_question("What is black body radiation?")

Question: What is black body radiation?
Destination: {'destination': 'physics'}
Answer: Blackbody radiation is a fundamental concept in physics that describes the emission of electromagnetic radiation by an object, such as a gas or a solid, when it is heated above absolute zero (0 Kelvin). The term "blackbody" comes from the idea that the object appears to absorb all wavelengths of visible light, regardless of their frequency.

In 1879, Max Planck proposed that the energy emitted by an ideal blackbody is quantized and distributed in discrete packets, now known as photons. This theory revolutionized our understanding of the behavior of radiation and laid the foundation for quantum mechanics.

The key features of blackbody radiation are:

1. **Quantization**: The energy of each photon is determined by Planck's constant (h) and the frequency of the emitted radiation.
2. **Discrete spectrum**: The visible light emitted by a blackbody has a discrete spectrum, meaning it consists of a series

In [25]:
await ask_question("What is 2 + 2?")

Question: What is 2 + 2?
Destination: {'destination': 'math'}
Answer: 2 + 2 = 4.


In [27]:
await ask_question("Why does every cell in our body contain DNA?")

Question: Why does every cell in our body contain DNA?
Destination: {'destination': 'default'}
Answer: Every cell in the human body contains DNA because it's essential for their function and survival. Here are some reasons why:

1. **Genetic information**: DNA contains the genetic instructions for the development, growth, and function of all living cells in the body. It provides the blueprint for creating proteins that perform specific functions, which is necessary for maintaining cellular homeostasis.
2. **Cellular identity**: DNA determines an organism's unique identity, including its characteristics, traits, and features. Each cell in the body has a distinct genetic makeup that sets it apart from other cells.
3. **Regeneration**: Cells can differentiate into different types of cells, which is essential for growth, repair, and replacement of damaged or dying cells. DNA plays a crucial role in this process by providing instructions for cell differentiation and tissue repair.
4. **Prot

In [28]:
await ask_question("Who is Napoleon Bonaparte?")

Question: Who is Napoleon Bonaparte?
Destination: {'destination': 'history'}
Answer: Napoleon Bonaparte was a French military and political leader who rose to power during the French Revolution. He is one of the most iconic figures in modern history, known for his military conquests, strategic genius, and ambitious ambitions.

Born on August 15, 1769, in Ajaccio, Corsica, Napoleon graduated from the École Militaire in Paris and became a second lieutenant in the French army at the age of 13. He quickly distinguished himself as a brilliant and courageous officer during the Italian campaign against the Austrians and the Kingdom of Sardinia.

In 1796, Napoleon was appointed commander of the French army in Italy, where he led the French to several victories, including the Battle of Arcola. However, his success was short-lived, as the Italian Republic collapsed, and he was forced to retreat.

In 1799, Napoleon seized power in a coup d'état, known as the Coup of 18 Brumaire (September 9), whi